In [34]:
library(tidyverse)
# library(gravity)

In [35]:
df <- read.csv("data/ky_grav_data.csv")

In [36]:
head(df)

,X,MIGPUMA,PUMA,filler_x,distance,year,internal,pop_orig,pop_dest,dens_orig,dens_dest,filler_y
,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>
1,0,21001002,21001001,124,0.0,2018,1,196280,196280,236.7249,236.7249,160
2,1,21001002,21002001,5,128602.4,2018,0,196280,107986,236.7249,130.2373,160
3,2,21001002,21003001,1,126902.1,2018,0,196280,107294,236.7249,129.4027,160
4,3,21001002,21005001,1,263705.0,2018,0,196280,131264,236.7249,242.3080,160
5,4,21001002,21006001,2,392468.0,2018,0,196280,104582,236.7249,126.1319,160
6,5,21001002,21007001,5,468036.7,2018,0,196280,103167,236.7249,124.4253,160


In [37]:
df_subset <- subset(df, internal != 1)
head(df_subset)

,X,MIGPUMA,PUMA,filler_x,distance,year,internal,pop_orig,pop_dest,dens_orig,dens_dest,filler_y
,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>
2,1,21001002,21002001,5,128602.4,2018,0,196280,107986,236.7249,130.2373,160
3,2,21001002,21003001,1,126902.1,2018,0,196280,107294,236.7249,129.4027,160
4,3,21001002,21005001,1,263705.0,2018,0,196280,131264,236.7249,242.3080,160
5,4,21001002,21006001,2,392468.0,2018,0,196280,104582,236.7249,126.1319,160
6,5,21001002,21007001,5,468036.7,2018,0,196280,103167,236.7249,124.4253,160
7,6,21001002,21009001,1,568326.8,2018,0,196280,119856,236.7249,144.5532,160


In [48]:
res <- lm(log(filler_x) ~ log(dens_orig) + log(dens_dest) + log(distance + 1) - 1, data=df_subset)

In [49]:
summary(res)


Call:
lm(formula = log(filler_x) ~ log(dens_orig) + log(dens_dest) + 
    log(distance + 1) - 1, data = df_subset)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.0240 -0.5346 -0.0433  0.6330  3.6645 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
log(dens_orig)     0.45788    0.04557  10.048  < 2e-16 ***
log(dens_dest)     0.15516    0.03557   4.362 1.65e-05 ***
log(distance + 1) -0.21347    0.02311  -9.237  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 0.8747 on 387 degrees of freedom
Multiple R-squared:  0.5702,	Adjusted R-squared:  0.5669 
F-statistic: 171.2 on 3 and 387 DF,  p-value: < 2.2e-16


In [40]:
min.RSS <- function(data, par) {
    with(data, sum(log(filler_x) - par[1] * log(pop_orig) - par[2] * log(pop_dest + par[3] * internal) - par[4] * log(distance+ 1))^2)
}

In [41]:
(result <- optim(par = c(1, 1, 3, -2), fn=min.RSS, data=df))

$par
[1]  0.8557589  1.0455801  3.1538942 -1.9118717

$value
[1] 1.734271e-05

$counts
function gradient 
     109       NA 

$convergence
[1] 0

$message
NULL

In [58]:
fit <- glm(filler_x / filler_y ~ log(dens_orig) + log(dens_dest) + log(distance + 1) - 1, family=quasipoisson(link = "log"), data=df_subset)

In [59]:
summary(fit)


Call:
glm(formula = filler_x/filler_y ~ log(dens_orig) + log(dens_dest) + 
    log(distance + 1) - 1, family = quasipoisson(link = "log"), 
    data = df_subset)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-0.25997  -0.11451  -0.04928   0.03525   1.09188  

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
log(dens_orig)     0.11889    0.04960   2.397  0.01701 *  
log(dens_dest)     0.12948    0.04300   3.011  0.00277 ** 
log(distance + 1) -0.42909    0.02286 -18.771  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for quasipoisson family taken to be 0.03881856)

    Null deviance: 694.1084  on 390  degrees of freedom
Residual deviance:   9.1499  on 387  degrees of freedom
AIC: NA

Number of Fisher Scoring iterations: 6
